<a href="https://colab.research.google.com/github/afhamash/Estimators/blob/main/EstimatorsMLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install -y git #install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!git config --global user.email "afhamash@gmail.com"
!git config --global user.name "Afham"

In [17]:
%cd /content/drive/MyDrive/Estimators


/content/drive/MyDrive/Estimators


In [21]:
import numpy as np


In [25]:
%cd /content/drive/MyDrive/Estimators
!git add .
!git commit -m "First commit"
!git push origin master

/content/drive/MyDrive/Estimators
[master (root-commit) 05ce770] First commit
 1 file changed, 1 insertion(+)
 create mode 100644 EstimatorsMLE.ipynb
fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [26]:
x =  5